In [22]:
import numpy as np
import csv
import time

np.random.seed(42)
def randomize(): np.random.seed(time.time())

In [23]:
RND_MEAN = 0.0
RND_STD = 0.003
LEARNING_RATE = 0.001

In [24]:
def abalone_execute(epochs=10, batch_size=10, report_period=1):
    load_abalone_dataset()
    train()
    return None

In [25]:
def load_abalone_dataset():
    with open('dataset/abalone.data.csv', 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
        
        global data, input_dim, output_dim
        input_dim, output_dim = 10, 1
        data = np.zeros([len(rows), input_dim+output_dim])

        for n, row in enumerate(rows):
            if row[0] == 'I': data[n, 0] = 1
            elif row[0] == 'M': data[n, 1] = 1
            elif row[0] == 'F': data[n, 2] = 1
            data[n, 3:] = row[1:]

In [26]:
def init_model():
    global weights, bias, input_dim, output_dim
    weights = np.random.normal(RND_MEAN, RND_STD, [input_dim, output_dim])
    bias = np.zeros([output_dim])

In [27]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

In [28]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_dim
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_dim], test_data[:, -output_dim:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_dim
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_dim], train_data[:, -output_dim:]

In [29]:
def run_train(x, y):

    #wx + b 계산
    output, aux_nn = forward_neuralnet(x)
    #(wx + b - y) ^ 2 계산
    loss, aux_pp = forward_postproc(output, y)
    # wx + b 와 y 간의
    accuracy = eval_accuracy(output, y)
    
    #dL/dL
    dL_dL = 1.0
    G_output = backprop_postproc(dL_dL, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

In [30]:
def forward_neuralnet(x):
    global weights, bias
    # x : 1 x 10
    # weights : 10 x 1
    # bias : 1 x 1
    output = np.matmul(x, weights) + bias
    return output, x

def backprop_neuralnet(G_upstream, x):
    global weights, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_upstream)
    G_b = np.sum(G_upstream, axis=0)

    weights -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

In [31]:
def forward_postproc(output, y):
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

def backprop_postproc(G_loss, diff):
    
    shape = diff.shape
    
    #dL_d((output-y)^2)
    g_loss_square = np.ones(shape) / np.prod(shape)
    #d((output-y)^2)_d(output-y)
    g_square_diff = 2 * diff
    #d(output-y)_d(output)
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    
    return G_output

In [32]:
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y)/y))
    return 1 - mdiff

In [33]:
def backprop_postproc_oneline(G_loss, diff):  # backprop_postproc() 대신 사용 가능
    return 2 * diff / np.prod(diff.shape)